In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.regularizers import l2
from tensorflow import keras
import numpy as np
from sklearn.metrics import classification_report

# Define data path
data_dir = '/kaggle/input/bengali-food-58/ALLFOOD'

# Get list of image files and labels
image_files = []
labels = []
for class_label in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_label)
    for image_file in os.listdir(class_dir):
        image_files.append(os.path.join(class_dir, image_file))
        labels.append(class_label)


# Split data into training and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(image_files, labels, test_size=0.1)

# Split training set into training and validation sets
train_files, valid_files, train_labels, valid_labels = train_test_split(train_files, train_labels, test_size=0.1)

# Create dataframes for the splits
train_df = pd.DataFrame({'filename': train_files, 'class': train_labels})
valid_df = pd.DataFrame({'filename': valid_files, 'class': valid_labels})
test_df = pd.DataFrame({'filename': test_files, 'class': test_labels})

In [ ]:
img_height = 224
img_width = 224
n_classes = 58

batch_size = 64

# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     rotation_range=20
   
    
# )
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.4,
    rotation_range=30,
    vertical_flip=True,
    horizontal_flip=True,
    width_shift_range=0.4,
    height_shift_range=0.3,
    brightness_range=[0.8, 1.2],
    channel_shift_range=20,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate training set from dataframe
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

# Generate validation set from dataframe
validation_generator = validation_datagen.flow_from_dataframe(
    valid_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

# Generate testing set from dataframe
test_generator = validation_datagen.flow_from_dataframe(
    test_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

In [ ]:
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input,Average

model_1=load_model('/kaggle/input/models/Xception_best_loss.hdf5')
model_1=Model(inputs=model_1.inputs,outputs=model_1.outputs,name='xception')

model_2=load_model('/kaggle/input/models/densenet_best_loss.hdf5')
model_2=Model(inputs=model_2.inputs,outputs=model_2.outputs,name='densenet')


models=[model_1,model_2]

model_input = Input(shape=(224,224,3))
model_outputs = [model(model_input) for model in models]

ensemble_output = Average()(model_outputs)

ensemble_model=Model(inputs=model_input,outputs=ensemble_output,name='ensemble')

ensemble_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
ensemble_model.summary()

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Get the class labels from the generator
class_labels = train_generator.classes

# Compute class weights
class_weights = compute_class_weight('balanced',classes= np.unique(class_labels),y= class_labels)


class_weights_dict = dict(zip(np.unique(class_labels), class_weights))

In [ ]:
from keras.callbacks import LearningRateScheduler
def learning_rate_scheduler(epoch, lr):
    initial_lr=0.0001
    warmup_epochs = 5 # Number of epochs for warm-up
    if epoch < warmup_epochs:
        # Gradually increase the learning rate
        warmup_factor = epoch / warmup_epochs
        lr = warmup_factor*initial_lr
        return lr
    else:
        return initial_lr

In [ ]:
lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
history=ensemble_model.fit(train_generator,
        steps_per_epoch = len(train_generator) ,
        validation_data = validation_generator,
        validation_steps = len(validation_generator),
        epochs=10,
        verbose=1, 
        callbacks=[ lr_scheduler],
         class_weight=class_weights_dict)

In [ ]:
y_pred = ensemble_model.predict(test_generator)
y_pred = tf.argmax(y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())
print('Classification Report:')
print(classification_report(test_generator.classes, y_pred, labels=range(len(class_labels)), target_names=class_labels))

In [ ]:
import json

# Assuming you have created the ensemble model by loading two separate models

# Save the weights of the ensemble model
ensemble_model.save_weights('ensemble_weights.h5')

# Save the architecture of the ensemble model
with open('ensemble_architecture.json', 'w') as f:
    f.write(ensemble_model.to_json())

# Alternatively, you can use YAML format instead of JSON
# with open('ensemble_architecture.yaml', 'w') as f:
#     f.write(ensemble_model.to_yaml())


In [ ]:
from tensorflow.keras.models import model_from_json

# Load the architecture from JSON
with open('ensemble_architecture.json', 'r') as f:
    ensemble_model_architecture = f.read()

# Alternatively, if using YAML
# with open('ensemble_architecture.yaml', 'r') as f:
#     ensemble_model_architecture = f.read()

# Create the ensemble model using the loaded architecture
model = model_from_json(ensemble_model_architecture)

# Load the weights into the ensemble model
model.load_weights('ensemble_weights.h5')


In [ ]:
data_df = pd.DataFrame({'filename': image_files, 'class': labels})

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1. / 255)
data_generator = validation_datagen.flow_from_dataframe(
    data_df,
    x_col='filename',
    y_col='class',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

In [ ]:
y_pred = model.predict(data_generator)
y_pred = tf.argmax(y_pred, axis=1)
class_labels = list(data_generator.class_indices.keys())
print('Classification Report:')
print(classification_report(data_generator.classes, y_pred, labels=range(len(class_labels)), target_names=class_labels))